In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from scipy.linalg import svd
from numpy import array
from numpy import diag
from numpy import dot
from numpy import zeros

In [3]:
df=pd.read_csv('./EEG_data.csv')
data = pd.read_csv('./demographic_info.csv')

In [4]:
data = data.rename(columns = {'subject ID': 'SubjectID',' gender':'gender',' age':'age',' ethnicity':'ethnicity'})
df = df.merge(data,how = 'inner',on = 'SubjectID')
df.head()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln,age,ethnicity,gender
0,0.0,0.0,56.0,43.0,278.0,301963.0,90612.0,33735.0,23991.0,27946.0,45097.0,33228.0,8293.0,0.0,0.0,25,Han Chinese,M
1,0.0,0.0,40.0,35.0,-50.0,73787.0,28083.0,1439.0,2240.0,2746.0,3687.0,5293.0,2740.0,0.0,0.0,25,Han Chinese,M
2,0.0,0.0,47.0,48.0,101.0,758353.0,383745.0,201999.0,62107.0,36293.0,130536.0,57243.0,25354.0,0.0,0.0,25,Han Chinese,M
3,0.0,0.0,47.0,57.0,-5.0,2012240.0,129350.0,61236.0,17084.0,11488.0,62462.0,49960.0,33932.0,0.0,0.0,25,Han Chinese,M
4,0.0,0.0,44.0,53.0,-8.0,1005145.0,354328.0,37102.0,88881.0,45307.0,99603.0,44790.0,29749.0,0.0,0.0,25,Han Chinese,M


In [5]:
df['gender']=df['gender'].replace({'M':1,'F':0})
df['ethnicity']=df['ethnicity'].replace({'Han Chinese':0,'Bengali':1,'English':2})
df.drop(columns = ['SubjectID','VideoID','predefinedlabel'],inplace=True)

In [6]:
top_fea=df.drop(['user-definedlabeln'],axis=1)

In [7]:
top_fea.shape

(12811, 14)

In [8]:
# Perform SVD
U, s, Vt = svd(top_fea)

# U, Sigma, and Vt are the matrices such that X = U * Sigma * Vt
# U: left singular vectors
# Sigma: singular values
# Vt: right singular vectors (transposed)

# Print the results
# print("Matrix U:")
# print(U)
# print("\nMatrix Sigma:")
# print(np.diag(Sigma))  # Convert singular values to a diagonal matrix for clarity
# print("\nMatrix Vt:")
# print(Vt)

In [9]:
s

array([1.02815735e+08, 2.47419719e+07, 1.17474564e+07, 6.78210125e+06,
       4.69828671e+06, 3.86504995e+06, 3.02495082e+06, 2.33043954e+06,
       6.60638244e+04, 6.60396203e+03, 1.69448609e+03, 8.88673211e+02,
       6.47587588e+01, 4.52827000e+01])

In [10]:
for i in range(0,14):
    print(s[i])

102815734.97584194
24741971.882308476
11747456.442044646
6782101.250381863
4698286.706368632
3865049.9485918744
3024950.821600757
2330439.5382966707
66063.82436054252
6603.962025051184
1694.4860899795544
888.6732111679671
64.75875879514957
45.282699976007116


In [11]:
s.sum()

160081352.55258036

In [12]:
for i in range (0,14):
    print(s[i]/s.sum())

0.6422717782951707
0.15455873834013067
0.07338429026695083
0.04236659137518352
0.02934936912670969
0.02414428593313115
0.018896334728351206
0.014557845127721632
0.00041268906907094747
4.125378702608129e-05
1.0585156003245183e-05
5.551384948950086e-06
4.0453655446145046e-07
2.828730470723225e-07


In [12]:
Sigma = zeros((top_fea.shape[0], top_fea.shape[1]))

(12811, 14)

In [14]:
Sigma[:top_fea.shape[1], :top_fea.shape[1]] = diag(s)
Sigma.shape


(12811, 14)

In [102]:
n_elements = 1
Sigma = Sigma[:, :n_elements]

Vt = Vt[:n_elements, :]

In [103]:
B = U.dot(Sigma.dot(Vt))
print("Reconstructed Matrix:")
print(B)

Reconstructed Matrix:
[[8.77049378e+00 1.08767932e+01 2.42723728e+01 ... 6.12931164e+00
  1.84488537e-02 1.79385242e-01]
 [2.15036547e+00 2.66679176e+00 5.95114411e+00 ... 1.50279567e+00
  4.52332318e-03 4.39819969e-02]
 [2.31933661e+01 2.87634257e+01 6.41877234e+01 ... 1.62088216e+01
  4.87875628e-02 4.74380085e-01]
 ...
 [1.91720724e+01 2.37763885e+01 5.30587789e+01 ... 1.33985166e+01
  4.03287166e-02 3.92131497e-01]
 [9.90568711e+00 1.22846117e+01 2.74140244e+01 ... 6.92264824e+00
  2.08367484e-02 2.02603653e-01]
 [3.92750299e+01 4.87072209e+01 1.08693785e+02 ... 2.74475878e+01
  8.26155627e-02 8.03302633e-01]]


In [104]:
B.shape

(12811, 14)

In [84]:
# scaler = StandardScaler()
# X=scaler.fit_transform(B)

In [105]:
from sklearn.model_selection import train_test_split
y=df['user-definedlabeln']
X_train,x_test,Y_train,y_test=train_test_split(X,y,random_state=108,test_size=0.27)

In [106]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [107]:
# Create an SVM classifier
svm_classifier = SVC(random_state=108)

# Train the classifier on the training data
svm_classifier.fit(X_train, Y_train)

SVC(random_state=108)

In [108]:
predictions = svm_classifier.predict(x_test).astype(int)
predictions

array([0, 1, 1, ..., 0, 0, 0])

In [109]:
accuracy = accuracy_score(y_test, predictions)
accuracy

0.5883203237930038

In [90]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import callbacks,layers

2023-11-14 23:26:38.810564: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [91]:
inputs = tf.keras.Input(shape=(X_train.shape[1]))
x = tf.keras.layers.Dense(256, activation='relu')(inputs)
x = tf.keras.layers.Dense(256, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model=tf.keras.Model(inputs,outputs)
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy',tf.keras.metrics.AUC(name='auc')]
             )
batch_size=32
epochs=50
history=model.fit(
            X_train,
            Y_train,
            validation_split=0.2,
            batch_size=batch_size,
            epochs=epochs,
            callbacks=[tf.keras.callbacks.ReduceLROnPlateau()]
)

2023-11-14 23:26:56.149098: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/50
234/234 [==============================] - 3s 11ms/step - loss: 0.6842 - accuracy: 0.5756 - auc: 0.5878 - val_loss: 0.6842 - val_accuracy: 0.5671 - val_auc: 0.5715 - lr: 0.0010
Epoch 2/50
234/234 [==============================] - 2s 9ms/step - loss: 0.6808 - accuracy: 0.5793 - auc: 0.5893 - val_loss: 0.6817 - val_accuracy: 0.5655 - val_auc: 0.5894 - lr: 0.0010
Epoch 3/50
234/234 [==============================] - 2s 9ms/step - loss: 0.6814 - accuracy: 0.5747 - auc: 0.5880 - val_loss: 0.6808 - val_accuracy: 0.5655 - val_auc: 0.5874 - lr: 0.0010
Epoch 4/50
234/234 [==============================] - 2s 10ms/step - loss: 0.6791 - accuracy: 0.5813 - auc: 0.5943 - val_loss: 0.6857 - val_accuracy: 0.5623 - val_auc: 0.5874 - lr: 0.0010
Epoch 5/50
234/234 [==============================] - 3s 13ms/step - loss: 0.6783 - accuracy: 0.5787 - auc: 0.5952 - val_loss: 0.6831 - val_accuracy: 0.5617 - val_auc: 0.5862 - lr: 0.0010
Epoch 6/50
234/234 [==============================] - 3s 11ms/

In [92]:
model.evaluate(x_test,y_test)

109/109 [==============================] - 1s 5ms/step - loss: 0.6754 - accuracy: 0.5875 - auc: 0.6054


[0.6753879189491272, 0.5874530076980591, 0.6054176092147827]

In [93]:
y_true = np.array(y_test)

y_pred = np.squeeze(model.predict(x_test))
y_pred = np.array(y_pred >=0.5, dtype=np.int)
y_pred

109/109 [==============================] - 1s 6ms/step


/tmp/ipykernel_31806/295032384.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_pred = np.array(y_pred >=0.5, dtype=np.int)


array([0, 0, 1, ..., 0, 0, 0])

In [94]:
accuracyKeras=accuracy_score(y_true, y_pred)
accuracyKeras

0.5874530211043654